<a href="https://colab.research.google.com/github/comradyo/kis/blob/main/lr-7/lr-7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками

## Задание 1.

В однопроцессорную  систему случайным образом поступают на выполнение $m$ разных типов программ, отличающихся известной трудоемкостью $Q_1, Q_2, \ldots, Q_m$. Входящий поток простейший с интенсивностью $\lambda$.

- Представив данную систему как одноканальную СМО с неограниченной очередью, вычислить среднее время обслуживания программ, считая длительность обслуживания случайной величиной (теоретически и экспериментально).

- Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ. Сравнить полученные результаты.

- Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет величина кванта на среднее время обслуживания программ.

In [346]:
Variant<-8
set.seed(Variant) 
m<-sample(c(6:20),1)
lambda_const<-runif(1,0.1, 2)
Q<-rexp(m,0.3)
q<-sample(c(1:4),1)
View(data.frame(m,q, lambda_const))
print(Q)

m,q,lambda_const
<int>,<int>,<dbl>
9,3,1.61935


[1] 1.012475452 3.263946321 1.459516694 2.855469094 1.489090714 2.516819748
[7] 0.299747437 4.973630433 0.003007615


#Часть 1:

Заданная система является системой вида $M/M/1/\infty$.

Среднее время обслуживания для данного вида систем:

$$ T_{сист} = \frac{1}{\mu(1-\rho)} = \frac{1}{\mu-\lambda} $$

Определим интенсивность обслуживания $\mu$:

$$ \mu = \frac{1}{Q_{ср}} =  \frac{m}{\sum_{i=1}^{m}Q_i} $$



Проведем теоретические рассчеты в среде R

In [347]:
mu <- m / sum(Q)
mu

# lambda > mu, уменьшим лямбду
lambda <- lambda_const - 1.2
lambda

1 / (mu - lambda)

[1] 0.503533

[1] 0.4193501

[1] 11.8789

Проведем экспериментальный расчет в среде R:

In [311]:
N <- 10000000   # Число тиков (прибавления дельт времени)
dt <- 0.01      # Тик (Дельта времени)
curTime <- 0    # Текущее время симуляции

buffer <- c() # буфер заявок в систему, обрабатывается первая заявка
bufferReceiveAt <- c() # буфер моментов времени, когда заявки появились в системе

numOfReqs <- 0
numOfSuccesfulReqs <- 0
timeInSystTotal <- 0
receiveRequestAt <- rexp(1, lambda)

getHandleDuration <- function() {
  #i <-sample(c(1:m),1)
  #return(rexp(1, 1/{Q[i]}))
  return (rexp(1, mu))
}

for (i in 1:N) {
  curTime <- curTime + dt

  if (curTime >= receiveRequestAt) {
    numOfReqs <- numOfReqs + 1
    bufferReceiveAt <- append(bufferReceiveAt, receiveRequestAt)
    handleDuration <- getHandleDuration()
    buffer <- append(buffer, handleDuration)
    receiveRequestAt <- receiveRequestAt + rexp(1, lambda)
  }

  # сокращение времени обработки текущей программы, которая находится на сервере.
  if (length(buffer) > 0) {
    buffer[1] <- buffer[1] - dt
    if (buffer[1] <= 0) {
      numOfSuccesfulReqs <- numOfSuccesfulReqs + 1
      timeSpentInSystem <- curTime - bufferReceiveAt[1]
      timeInSystTotal <- timeInSystTotal + timeSpentInSystem
      buffer <- buffer[-1]
      bufferReceiveAt <- bufferReceiveAt[-1]
    }
  }
}

timeInSystTotal / numOfSuccesfulReqs

[1] 11.47562

#Часть 2:



In [136]:
N <- 10000000  # Число тиков (прибавления дельт времени)
dt <- 0.1      # Тик (Дельта времени)
curTime <- 0   # Текущее время симуляции

buffer <- c() # буфер заявок в систему, обрабатывается первая заявка
bufferReceiveAt <- c() # буфер моментов времени, когда заявки появились в системе

numOfReqs <- 0
numOfSuccesfulReqs <- 0
timeInSystTotal <- 0
receiveRequestAt <- rexp(1, lambda)

getHandleDuration <- function() {
  #i <-sample(c(1:m),1)
  #return(rexp(1, 1/{Q[i]}))
  return (rexp(1, mu))
}

for (i in 1:N) {
  curTime <- curTime + dt

  if (curTime >= receiveRequestAt) {
    numOfReqs <- numOfReqs + 1
    bufferReceiveAt <- append(bufferReceiveAt, receiveRequestAt)
    handleDuration <- getHandleDuration()
    buffer <- append(buffer, handleDuration)

    # поддержка отсортированности списка
    j <- length(buffer)
    while(j > 1 && buffer[j] < buffer[j-1]) {
      a <- buffer[j-1]
      buffer[j-1] <- buffer[j]
      buffer[j] <- a

      b <- bufferReceiveAt[j-1]
      bufferReceiveAt[j-1] <- bufferReceiveAt[j]
      bufferReceiveAt[j] <- b

      j <- j - 1
    }

    receiveRequestAt <- receiveRequestAt + rexp(1, lambda)
  }

  # сокращение времени обработки текущей программы, которая находится на сервере.
  if (length(buffer) > 0) {
    buffer[1] <- buffer[1] - dt
    if (buffer[1] <= 0) {
      numOfSuccesfulReqs <- numOfSuccesfulReqs + 1
      timeSpentInSystem <- curTime - bufferReceiveAt[1]
      timeInSystTotal <- timeInSystTotal + timeSpentInSystem
      buffer <- buffer[-1]
      bufferReceiveAt <- bufferReceiveAt[-1]
    }
  }
}

timeInSystTotal / numOfSuccesfulReqs

[1] 5.699725

#Часть 3:



In [343]:
RR <- function(q) {
  N <- 10000000    # Число тиков (прибавления дельт времени)
  dt <- 0.01       # Тик (Дельта времени)
  curTime <- 0     # Текущее время симуляции

  buffer <- c() # буфер заявок в систему, обрабатывается первая заявка
  bufferReceiveAt <- c() # буфер моментов времени, когда заявки появились в системе

  numOfReqs <- 0
  numOfSuccesfulReqs <- 0
  timeInSystTotal <- 0
  receiveRequestAt <- rexp(1, lambda)

  getHandleDuration <- function() {
    #i <-sample(c(1:m),1)
    #return(rexp(1, 1/{Q[i]}))
    return (rexp(1, mu))
  }

  qCounter <- q       # (1 / dt) = количество тиков, находящихся в единице времени

  for (i in 1:N) {
    curTime <- curTime + dt

    if (curTime >= receiveRequestAt) {
      numOfReqs <- numOfReqs + 1
      bufferReceiveAt <- append(bufferReceiveAt, receiveRequestAt)
      handleDuration <- getHandleDuration()
      buffer <- append(buffer, handleDuration)
      receiveRequestAt <- receiveRequestAt + rexp(1, lambda)
    }

    # обработка текущей программы, которая находится на сервере.
    if (length(buffer) > 0) {
      buffer[1] <- buffer[1] - dt
      qCounter <- qCounter - dt
      if (buffer[1] <= 0) {
        numOfSuccesfulReqs <- numOfSuccesfulReqs + 1
        timeSpentInSystem <- curTime - bufferReceiveAt[1]
        timeInSystTotal <- timeInSystTotal + timeSpentInSystem
        buffer <- buffer[-1]
        bufferReceiveAt <- bufferReceiveAt[-1]

        qCounter <- q # reset счетчика
      } else if (qCounter <= 0) { # заявка не обработалась, ставим её в конец очереди
          buffer <- append(buffer, buffer[1])
          buffer <- buffer[-1]

          bufferReceiveAt <- append(bufferReceiveAt, bufferReceiveAt[1])
          bufferReceiveAt <- bufferReceiveAt[-1]

          qCounter <- q
      }
    }
  }

  return(timeInSystTotal / numOfSuccesfulReqs)
}

In [344]:
for (q in c(0.01, 0.1, 0.5, 1, 3, 5, 8, 10, 20, 30)) {
  cat(q,"\t", RR(q),'\n')
}

0.01 	 11.64731 
0.1 	 13.08243 
0.5 	 11.31081 
1 	 11.98668 
3 	 10.89952 
5 	 11.23136 
8 	 11.02087 
10 	 11.48487 
20 	 11.57323 
30 	 11.24899 


Решение для заранее заданного массива поступления заявок и длительностей их обработки:

In [358]:
getHandleDuration <- function() {
  #i <-sample(c(1:m),1)
  #return(rexp(1, 1/{Q[i]}))
  return (rexp(1, mu))
}

numOfReqs <- 10000
buff <- double(numOfReqs) # буфер заявок в систему, обрабатывается первая заявка
for (i in 1:numOfReqs) {
  buff[i] <- getHandleDuration()
}
buffRecAt <- double(numOfReqs) # буфер моментов времени, когда заявки появились в системе
buffRecAt[1] <- rexp(1, lambda)
for (i in 2:numOfReqs) {
  buffRecAt[i] <- buffRecAt[i-1] + rexp(1, lambda)
}

N <- 1000000

RRAltVersion <- function(q, bf, bfReceiveAt) {
  dt <- 0.01      # Тик (Дельта времени)
  curTime <- 0    # Текущее время симуляции

  i <- 1
  timeInSystemTotal <- 0

  buffer <- c()
  bufferReceiveAt <- c()

  nHandled <- 0
  qCounter <- q   # (1 / dt) = количество тиков, находящихся в единице времени

  for(i in 1:N) {
    curTime <- curTime + dt
    if (curTime >= bfReceiveAt[1]) {
      bufferReceiveAt <- append(bufferReceiveAt, bfReceiveAt[1])
      buffer <- append(buffer, bf[1])
      bfReceiveAt <- bfReceiveAt[-1]
      bf <- bf[-1]
    }
    if(length(buffer) > 0) {
      buffer[1] <- buffer[1] - dt
      qCounter <- qCounter - dt
      if (buffer[1] <= 0) {
        nHandled <- nHandled + 1

        timeInSystemTotal <- timeInSystemTotal + curTime - bufferReceiveAt[1]
        buffer <- buffer[-1]
        bufferReceiveAt <- bufferReceiveAt[-1]

        qCounter <- q # reset счетчика
      } else if (qCounter <= 0) { # заявка не обработалась, ставим её в конец очереди
          buffer <- append(buffer, buffer[1])
          buffer <- buffer[-1]

          bufferReceiveAt <- append(bufferReceiveAt, bufferReceiveAt[1])
          bufferReceiveAt <- bufferReceiveAt[-1]

          qCounter <- q
      }
    }
  }

  return(timeInSystemTotal / nHandled)
}

for (q in c(0.01, 0.1, 0.5, 1, 3, 5, 8, 10, 20, 30, 100)) {
  bf <- buff
  bfRecAt <- buffRecAt
  cat(q,"\t", RRAltVersion(q, bf, bfRecAt),'\n')
}

0.01 	 10.6506 
0.1 	 10.65244 
0.5 	 10.6702 
1 	 10.68152 
3 	 10.64463 
5 	 10.60341 
8 	 10.64998 
10 	 10.59512 
20 	 10.5726 
30 	 10.5726 
100 	 10.5726 


Решение для фиксированного массива (без прибытия дополнительных заявок):

In [333]:
getHandleDuration <- function() {
  i <-sample(c(1:m),1)
  return(rexp(1, 1/{Q[i]}))
  #return (rexp(1, mu))
}

numOfReqs <- 100
buff <- double(numOfReqs) # буфер заявок в систему, обрабатывается первая заявка
for (i in 1:numOfReqs) {
  buff[i] <- getHandleDuration()
}
buffRecAt <- double(numOfReqs) # буфер моментов времени, когда заявки появились в системе
buffRecAt[1] <- rexp(1, lambda)
for (i in 2:numOfReqs) {
  buffRecAt[i] <- buffRecAt[i-1] + rexp(1, lambda)
}

RRWithLimitedNumOfReqs <- function(q, buffer, bufferReceiveAt) {
  dt <- 0.01      # Тик (Дельта времени)
  curTime <- 0    # Текущее время симуляции

  i <- 1
  nReqs <- length(buffer)
  timeInSystemTotal <- 0

  nHandled <- 0

  while(nHandled != nReqs) {
    while(curTime < bufferReceiveAt[1]) {
      curTime <- curTime + dt
    }
    qCounter <- q   # (1 / dt) = количество тиков, находящихся в единице времени
    while(qCounter > 0) {
      buffer[1] <- buffer[1] - dt
      qCounter <- qCounter - dt
      if (buffer[1] <= 0) {
        nHandled <- nHandled + 1

        timeInSystemTotal <- timeInSystemTotal + curTime - bufferReceiveAt[1]
        buffer <- buffer[-1]
        bufferReceiveAt <- bufferReceiveAt[-1]

        qCounter <- 0 # reset счетчика
      } else if (qCounter <= 0) { # заявка не обработалась, ставим её в конец очереди
          buffer <- append(buffer, buffer[1])
          buffer <- buffer[-1]

          bufferReceiveAt <- append(bufferReceiveAt, bufferReceiveAt[1])
          bufferReceiveAt <- bufferReceiveAt[-1]

          qCounter <- 0
      }
      curTime <- curTime + dt
    }
  }

  return(timeInSystemTotal / nReqs)
}

In [334]:
for (q in c(0.01, 0.1, 0.5, 1, 3, 5, 8, 10, 20, 30, 100)) {
  cat(q,"\t", RRWithLimitedNumOfReqs(q, buff, buffRecAt),'\n')
}

0.01 	 165.5326 
0.1 	 144.5221 
0.5 	 113.2033 
1 	 81.85206 
3 	 36.74816 
5 	 16.98206 
8 	 14.20136 
10 	 14.73666 
20 	 11.50396 
30 	 11.50396 
100 	 11.50396 
